<img src="Imgs/swap.png">

But it gets even better! Imagine now _Alice_ and _Bob_ share an entangled pair, _Alice_ and _Charlie_ share an entangled pair, and _Charlie_ and _Diane_ share an entangled pair. So, we have 6 qubits in total and they are paired off. _Alice_ and _Bob_ perform the teleportation protocol, as do _Charlie_ and _Diane_.   


_Alice_ and _Charlie_ measure and hence collapse each of the qubits they had initially entangled. Those qubits are individually teleported to _Bob_ and _Diane_, respectively. But, _Bob_ and _Diane_ now share entanglement!   



This protocol is called entanglement swapping and it is very useful in quantum networking scenarios.

In [1]:
#Codes in this notebook are inspired by IBM's Qiskit tutorials and the arXiv:1903.04359v1 paper

#Libraries needed to implement and simulate quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute 
import math as m
#Custem functions to simplify answers
import Our_Qiskit_Functions as oq #a part of the libabry presented in arXiv:1903.04359v1.


#Initialize backends simulators to visualize circuits
S_simulator = Aer.backends(name='statevector_simulator')[0]
Q_simulator = Aer.backends(name='qasm_simulator')[0]
sim = Aer.get_backend('aer_simulator')

#Create quantum registers (to hold qubits)
q = QuantumRegister(2)
#Classical register to hold classical bits (Used to measure results)
c = ClassicalRegister(2)
#Create a quantum circuit using the above registers
qc = QuantumCircuit(q,c)

## The Bell States
---

There are four states of two qubits which lead to this maximal value of $2\sqrt{2}$ and they are known as the four maximally entangled two-qubit states or Bell states.    


$\begin{aligned}
\left|\Phi_{+}\right\rangle=\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)
\\
\left|\Phi_{-}\right\rangle=\frac{1}{\sqrt{2}}(|00\rangle-|11\rangle)
\\
\left|\Psi_{+}\right\rangle=\frac{1}{\sqrt{2}}(|01\rangle+|10\rangle)
\\
\left|\Psi_{-}\right\rangle=\frac{1}{\sqrt{2}}(|01\rangle-|10\rangle)
\end{aligned}$

In [2]:
#Creating Phi+
qc.h(q[0])
qc.cx(q[0],q[1])
qc.draw(output = "mpl")

┌───┐     
q0_0: ┤ H ├──■──
      └───┘┌─┴─┐
q0_1: ─────┤ X ├
           └───┘
c0: 2/══════════

In [3]:
#Creating Phi-
qc.h(q[0])
qc.cx(q[0],q[1])
qc.z(q[1])
qc.draw(output = "mpl")

┌───┐     ┌───┐          
q0_0: ┤ H ├──■──┤ H ├──■───────
      └───┘┌─┴─┐└───┘┌─┴─┐┌───┐
q0_1: ─────┤ X ├─────┤ X ├┤ Z ├
           └───┘     └───┘└───┘
c0: 2/═════════════════════════

In [4]:
#Creating Psi+
qc.h(q[0])
qc.cx(q[0],q[1])
qc.x(q[1])
qc.draw(output = "mpl")

┌───┐     ┌───┐     ┌───┐          
q0_0: ┤ H ├──■──┤ H ├──■──┤ H ├──■───────
      └───┘┌─┴─┐└───┘┌─┴─┐├───┤┌─┴─┐┌───┐
q0_1: ─────┤ X ├─────┤ X ├┤ Z ├┤ X ├┤ X ├
           └───┘     └───┘└───┘└───┘└───┘
c0: 2/═══════════════════════════════════

In [5]:
#Creating Psi-
qc.h(q[0])
qc.cx(q[0],q[1])
qc.z(q[1])
qc.x(q[1])
qc.draw(output = "mpl")

┌───┐     ┌───┐     ┌───┐     ┌───┐               
q0_0: ┤ H ├──■──┤ H ├──■──┤ H ├──■──┤ H ├──■────────────
      └───┘┌─┴─┐└───┘┌─┴─┐├───┤┌─┴─┐├───┤┌─┴─┐┌───┐┌───┐
q0_1: ─────┤ X ├─────┤ X ├┤ Z ├┤ X ├┤ X ├┤ X ├┤ Z ├┤ X ├
           └───┘     └───┘└───┘└───┘└───┘└───┘└───┘└───┘
c0: 2/══════════════════════════════════════════════════

## Bell state analyzer
---

Suppose we have an arbitrary Bell state and we want to know which one it is. Some systems (e.g., when we are working with photons) can directly detect _some_ Bell states. For memory-based qubit systems, we can figure out which Bell state we have by using a CNOT and a Hadamard gate, then measuring the state in the computational (z-basis) basis. If the Bell state is good, then doing so will always give the same pattern that we can then use to classify the input Bell state. That's the job of a Bell state analyzer. The table below shows how to map from the measurement outputs to which one of the Bell states we had.


| Bell state| q0,q1  |
|---------  |--------------       |
| $\left|\Phi_{+}\right\rangle$         | 0,0       |
| $\left|\Phi_{-}\right\rangle$         | 0,1       |
| $\left|\Psi_{+}\right\rangle$         | 1,0       |
| $\left|\Psi_{-}\right\rangle$         | 1,1       |

In [20]:
#Measure state Phi+
q = QuantumRegister(2)
#Classical register to hold classical bits (Used to measure results)
c = ClassicalRegister(2)
#Create a quantum circuit using the above registers
qc = QuantumCircuit(q,c)

#Create the state.
# EXERCISE: Pick one of the Bell states, put in the two, three or four gates to create it.
# EXERCISE: After you have done this with the Bell states, try it with |00>, |01>, |10>, and |11>
# as inputs. What are the measurement results you get?  Can you map from that back to the original state?
#qc.h(q[0])
#qc.cx(q[0],q[1])
qc.x(q[0])

# let's see what the state is
print(oq.Wavefunction(qc))

#measure in the Bell basis
qc.cx(q[0],q[1])
qc.h(q[0])
qc.measure(q,c)

# Compile and run the Quantum circuit on a simulator backend
job_sim = execute(qc, Q_simulator)
sim_result = job_sim.result()
M = sim_result.get_counts(qc)
print(M)
qc.draw(output = "mpl")

1.0  |10>   
{'11': 505, '10': 519}


┌───┐     ┌───┐┌─┐
q184_0: ┤ X ├──■──┤ H ├┤M├
        └───┘┌─┴─┐└┬─┬┘└╥┘
q184_1: ─────┤ X ├─┤M├──╫─
             └───┘ └╥┘  ║ 
 c14: 2/════════════╩═══╩═
                    1   0

## Entanglement Swapping
---

Let's say we have three people, Alice, Bob and Clara. Alice and Bob each create a Bell state, and send one of their two qubits to Clara. We will emulate this structure by giving Alice and Bob each a two-qubit quantum register, and Clara a classical register, in the circuit below.


Clara then performs a Bell state analyzing operation and sends the resulting two classical bits to Alice or Bob to apply on their remaining qubit. 


This results is used to select which unitary (x, z) is used on, say Bob's, initial qubit. Doing so, entangling Bob's qubit with Alice's initial one.

In [22]:
#Create the initial state
alice = QuantumRegister(2)
bob = QuantumRegister(2)
clara = ClassicalRegister(2)
qc = QuantumCircuit(alice, bob, clara)


# Put Alice's qubits in Phi+ state
qc.h(alice[0])
qc.cx(alice[0],alice[1])

#Put Bob's qubits in Phi+ state
qc.h(bob[0])
qc.cx(bob[0],bob[1])

# Next, in the real world, Alice and Bob would each send a qubit to Clara.
# Clara applies Bell state analyzer to alice[1] and bob[0]
qc.cx(alice[1],bob[0])
qc.h(bob[0])

#Clara measures the qubits to complete the BSA
qc.measure(alice[1],clara[0])
qc.measure(bob[0],clara[1])

# Now Clara would send the classical bits to Bob, which he can use to correct his qubit, after
# which Alice and Bob should be left sharing a Bell pair, even though Alice and Bob have never
# directly interacted.
#Bob perform unitary transportation
if clara[0] == 1:
    qc.z(bob[1])
# If needed Perform a bit flip correction to qubit (3)
if clara[1] == 1:
    qc.x(bob[1])

# EXERCISE: Add code to print out the wave function on the remaining two qubits. Is it a Bell pair?
# EXERCISE: Add code to measure out the two remaining qubits. Do the results match what you expect?
    